In [1]:
import os
import numpy as np
import keras.backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler,Normalizer
import pandas as pd
import sklearn.svm as svm
import xgboost
from imblearn.datasets import make_imbalance
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix

df = pd.DataFrame(columns=['num_features','fold1','fold2','fold3','fol4'])
#for i in range(1,80):
# for i in range(1,25):
# Accuracy = np.array(i)
Accuracy = np.array('Accuracy')
Prec = np.array('Precision')
Sens = np.array('Sensitivity')
Spec = np.array('Specificity')
for fold in range(1,5):
    features = pd.read_csv('ICHFeatures.csv',header=0)
    OS_train = pd.read_csv('Fold'+str(fold)+'/Train_dir.txt',header = None, dtype=str)
    OS_train.columns = ['ID','OS']
    OS_valid = pd.read_csv('Fold'+str(fold)+'/Val_Dir.txt',header = None, dtype=str)
    OS_valid.columns = ['ID','OS']
    features['ID']=features['ID'].str.replace('ct1','')
    train = pd.merge(features, OS_train, how='right', on='ID')
    test =  pd.merge(features, OS_valid, how='right', on='ID')
    norm_wihtout = [col for col in train.columns if col not in ['ID','GCS','Onset','OS']]
    #norm_valid = [col for col in test.columns if col not in ['ID','GCS','Onset','OS']]
    scaler = StandardScaler()
    train_ss = scaler.fit_transform(train[norm_wihtout])
    test_ss = scaler.transform(test[norm_wihtout])
    train[norm_wihtout] = train_ss
    test[norm_wihtout] = test_ss
    #train = train.assign(norm_train.values = train_ss)
    col_withoutID = [col for col in train.columns if col not in ['ID','OS']]
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(train[col_withoutID], train['OS'].values.astype(int))
    train_X = pd.DataFrame(X_res,columns = col_withoutID)
    train_y = pd.DataFrame(y_res, columns = ['OS'])
    num_features = 11
    estimator = xgboost.XGBClassifier(max_depth=10, learning_rate=0.1)
    print(num_features)
    rfe=RFE(estimator,  n_features_to_select=num_features,step=1)
    rfe.fit(train_X,train_y)
    ranking_RFE=rfe.ranking_
    indices=np.where(ranking_RFE==1)
    indices = list(indices[0])
    data_RFE=train_X.iloc[:,indices]
    valid_RFE = test[col_withoutID].iloc[:,indices]
    print(data_RFE.columns)
    model = xgboost.XGBClassifier(max_depth=10, learning_rate=0.1)
    model.fit(data_RFE, train_y)

    Y_pred=model.predict(valid_RFE).ravel()
    acc=metrics.accuracy_score(test['OS'].values.astype(int),Y_pred)
    print("accuracy score = "+str(acc)) 
    Accuracy = np.append(Accuracy,acc)
    con_matrix = confusion_matrix(test['OS'].values.astype(int).tolist(),Y_pred.tolist())
    TN,FP,FN,TP = con_matrix.ravel()
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    #Precision
    PPV = TP/(TP+FP)
    Prec = np.append(Prec,PPV)
    Sens = np.append(Sens,TPR)
    Spec = np.append(Spec,TNR)
    
Accuracy = pd.DataFrame(data = Accuracy.reshape(1,5),columns = df.columns)
Prec = pd.DataFrame(data = Prec.reshape(1,5),columns = df.columns)
Sens = pd.DataFrame(data = Sens.reshape(1,5),columns = df.columns)
Spec = pd.DataFrame(data = Spec.reshape(1,5),columns = df.columns)
df = df.append(Accuracy)
df = df.append(Prec)
df = df.append(Sens)
df= df.append(Spec)

Using TensorFlow backend.


11


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Index(['Age', 'Onset', 'CTA', 'Centroid1', 'FirstAxis2', 'SecondAxis2',
       'SecondAxis3', 'bb5', 'entropy', 'diameter', 'f1'],
      dtype='object')
accuracy score = 0.6
11


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Index(['Age', 'CTA', 'Centroid3', 'MajorAxisLength', 'SecondAxis3', 'kurtosis',
       'bb1', 'bb5', 'f1', 'f2', 'f5'],
      dtype='object')
accuracy score = 0.85
11


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Index(['Age', 'CTA', 'Centroid1', 'Centroid3', 'MajorAxisLength', 'FirstAxis3',
       'kurtosis', 'bb2', 'bb4', 'bb5', 'solidity'],
      dtype='object')
accuracy score = 0.85


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
df.to_csv('xgb_results.csv')

In [ ]:
import shap
import matplotlib.pyplot as pl
shap_values = shap.TreeExplainer(model).shap_values(data_RFE) 
fig = shap.summary_plot(shap_values, data_RFE, show=False)
plt.savefig('shap_xgboost.pdf')

In [14]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = metrics.roc_curve(test['OS'].values.astype(int).reshape(20,1), Y_pred.reshape(20,1), pos_label=1)

In [16]:
tpr

array([0. , 0.6, 1. ])

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = 2
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(test['OS'].values.astype(int).reshape(20,1)[:, i], (Y_pred.reshape(20,1))[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

IndexError: index 1 is out of bounds for axis 1 with size 1

In [12]:
test['OS'].values.astype(int).reshape(20,1)

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1]])

In [9]:
Y_pred.reshape(20,1)

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1]])